## Assignment 1
### Name:T.AMRUTHAVARSHINI
### Roll Number:21CS30055

In [70]:
# import all the necessary libraries here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score, precision_score

In [71]:
df = pd.read_csv('../../dataset/decision-tree.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [72]:
train_size = int(0.6 * len(df))#Taking train as 0.6 , validation as 0.2 , test_size as 0.2 of tital data
validation_size = int(0.2 * len(df))
test_size = len(df) - train_size - validation_size

train_data = df.sample(n=train_size, random_state=42)
remaining_data = df.drop(train_data.index)
validation_data = remaining_data.sample(n=validation_size, random_state=42)
test_data = remaining_data.drop(validation_data.index)


In [73]:
attributes = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']


In [74]:
class TreeNode:  #Defining Tree Class
    def __init__(self, attribute=None, value=None, classification=None):
        self.attribute = attribute  
        self.value = value          
        self.classification = classification  
        self.children = {}     

In [75]:
def cal_entropy(data):  #Calculate Entropy
    class_count = data['Outcome'].value_counts() #returns number od 0 and 1 in outcome
    total_instances = len(data)
    entropy =0
    for count in class_count:
        prob = count/total_instances    
        entropy = entropy-(prob*math.log2(prob))
  
    return entropy

In [76]:
def choose_attribute(data,attributes): #Choose best attribute
    base_entropy = cal_entropy(data)
    best_information_gain = 0
    best_attribute = None
    
    for attribute in attributes:
        grouped = data.groupby(attribute)
        weighted_entropy = 0
        
        for value,group in grouped:
            subset_size = len(group)
            entropy  = cal_entropy(group)
            weighted_entropy += (subset_size / len(data))*entropy
            
        information_gain = base_entropy - weighted_entropy  #Difference between entropy is information gain
        
        if information_gain > best_information_gain:
            best_information_gain = information_gain
            best_attribute = attribute
    attributes.remove(best_attribute)  
            
    return best_attribute
    

In [77]:
def Tree(df,attributes):
    class_count = df['Outcome'].value_counts()
    if len(class_count) == 1:
        return TreeNode(classification = class_count.idxmax()) #getting the index of maximum value in data i,e return most common class as all are same class
    
    if len(df) <10:
        dominant_class = class_count.idxmax()
        return TreeNode(classification=dominant_class)
    
    if len(attributes) ==0:
        dominant_class = class_count.idxmax()
        return TreeNode(classification=dominant_class)
        
    best_attribute = choose_attribute(df,attributes)
    node = TreeNode(attribute=best_attribute)

    grouped = df.groupby(best_attribute) 
    for value,group in grouped:
        if len(group)==0:
            dominant_class = class_counts.idxmax()
            node.children[value] = TreeNode(classification = dominant_class)
        else:
            new_attributes = [attr for attr in attributes if attr!= best_attribute]
            node.children[value] = Tree(group,new_attributes)
    return node

In [78]:
root_node = Tree(train_data,attributes)

In [79]:
def print_tree(node, level=0):
    if node.attribute is not None:
        print("  " * level + f"Attribute: {node.attribute}")
        print("  " * level + f"Value: {node.value}")
    if node.classification is not None:
        print("  " * level + f"Class: {node.classification}")
        return
    for value, child_node in node.children.items():
        print("  " * level + f"Value: {value}")
        print_tree(child_node, level + 1)


print_tree(root_node)


Attribute: DiabetesPedigreeFunction
Value: None
Value: 0.078
  Class: 0
Value: 0.085
  Class: 0
Value: 0.08800000000000001
  Class: 1
Value: 0.092
  Class: 0
Value: 0.1
  Class: 0
Value: 0.102
  Class: 0
Value: 0.107
  Class: 0
Value: 0.10800000000000001
  Class: 0
Value: 0.115
  Class: 0
Value: 0.121
  Class: 0
Value: 0.122
  Class: 0
Value: 0.127
  Class: 1
Value: 0.128
  Class: 1
Value: 0.129
  Class: 1
Value: 0.133
  Class: 0
Value: 0.134
  Class: 0
Value: 0.135
  Class: 1
Value: 0.13699999999999998
  Class: 1
Value: 0.14
  Class: 0
Value: 0.141
  Class: 1
Value: 0.142
  Class: 0
Value: 0.14300000000000002
  Class: 0
Value: 0.14800000000000002
  Class: 0
Value: 0.149
  Class: 0
Value: 0.15
  Class: 1
Value: 0.153
  Class: 1
Value: 0.154
  Class: 0
Value: 0.156
  Class: 0
Value: 0.157
  Class: 0
Value: 0.158
  Class: 0
Value: 0.159
  Class: 0
Value: 0.16
  Class: 0
Value: 0.162
  Class: 0
Value: 0.16399999999999998
  Class: 0
Value: 0.165
  Class: 1
Value: 0.166
  Class: 0
Value: 0.

In [80]:
def calc_accuracy(node,test_data):
    correct_pred = 0
    total_instances = len(test_data)
    
    for _, instance in test_data.iterrows():
        if predict(instance,node):
            correct_pred +=1
    return correct_pred / total_instances
    

In [81]:
def calculate_accuracy(node, validation_data):
    correct_predictions = 0
    total_predictions = len(validation_data)

    for _, row in validation_data.iterrows():
        if predict(node, row) == row['Outcome']:  
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

def predict(node, data_row):
    if node.classification is not None:
        return node.classification
    
    attribute_value = data_row[node.attribute]
    
    if attribute_value in node.children:
        child_node = node.children[attribute_value]
        return predict(child_node, data_row)
    else:
        
        return None 



In [83]:
def reduced_error_pruning(node, validation_data):
    if node is None:
        return node
    if node.classification is not None:
        return node

    original_children = node.children.copy()
    for value, child in original_children.items():
        node.children[value] = reduced_error_pruning(child, validation_data)

    validation_accuracy = calculate_accuracy(node, validation_data) #if calculted accuracy is more return node
    node.children = {} 
    pruned_accuracy = calculate_accuracy(node, validation_data)
    if validation_accuracy >= pruned_accuracy:
        return node
    else:
        return original_children  
    
pruned_tree = reduced_error_pruning(root_node, validation_data)


def print_pruned_tree(node, level=0):
    if node is None:
        return
    
    indent = "  " * level
    if node.attribute:
        print(f"{indent}Attribute: {node.attribute}")
    
    if node.value:
        print(f"{indent}Value: {node.value}")
    
    if node.classification:
        print(f"{indent}Classification: {node.classification}")
    
    for value, child in node.children.items():
        print_pruned_tree(child, level + 1)

# Printing the pruned tree
print_pruned_tree(pruned_tree)

Attribute: DiabetesPedigreeFunction


In [69]:

def calculate_metrics(y_true, y_pred):  #calculating accuracy,precision,recalls
    num_classes = len(np.unique(y_true))
    class_accuracies = []
    class_precisions = []
    class_recalls = []

    for class_label in range(num_classes):
        tp = np.sum((y_true == class_label) & (y_pred == class_label))
        fp = np.sum((y_true != class_label) & (y_pred == class_label))
        fn = np.sum((y_true == class_label) & (y_pred != class_label))

        accuracy =  np.sum(y_true == class_label) / len(y_true)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0

        class_accuracies.append(accuracy)
        class_precisions.append(precision)
        class_recalls.append(recall)

    macro_accuracy = np.mean(class_accuracies)
    macro_precision = np.mean(class_precisions)
    macro_recall = np.mean(class_recalls)

    return macro_accuracy, macro_precision, macro_recall


true_labels = validation_data['Outcome'].values  
predicted_labels = []
for _, row in validation_data.iterrows():
    predicted_label = predict(pruned_tree, row)  
    predicted_labels.append(predicted_label)


predicted_labels = np.array(predicted_labels)


macro_accuracy, macro_precision, macro_recall = calculate_metrics(true_labels, predicted_labels)


#print("Macro Accuracy:", macro_accuracy)
#print("Macro Precision:", macro_precision)
#print("Macro Recall:", macro_recall)
